In [79]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1636\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [80]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [81]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("C:/Users/Lenovo/OneDrive/سطح المكتب/week 4/day 1/lab1/lab-natural-language-processing/your-code/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [82]:
print(data.head())

                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0
3  Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                fyi      0


### Let's divide the training and test set into two partitions

In [83]:
# Your code
from sklearn.model_selection import train_test_split

# Assuming `data` is your DataFrame
X = data['text']  # Features
y = data['label']  # Labels

# Splitting the data into training and test sets (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting sets
print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")



Training set: (800,), (800,)
Test set: (200,), (200,)


## Data Preprocessing

In [84]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [85]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [86]:
# Your code
from bs4 import BeautifulSoup
import re
def clean_html(text):
    # Remove inline JavaScript and CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    # Use BeautifulSoup to remove remaining HTML tags
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    return text.strip()
# Example usage
sample_html = "<html><head><style>body {font-size: 12px;}</style></head><body><!-- Comment --><p>This is a <b>test</b>.</p><script>console.log('Hello');</script></body></html>"
cleaned_text = clean_html(sample_html)
print(cleaned_text)



This is a test.


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [87]:
# Your code
import re
def clean_text(text):
   
    text = re.sub(r'[^a-zA-Z\s]', '', text)
  
    text = re.sub(r'\d+', '', text)

    text = re.sub(r'\b\w\b', '', text)

    text = re.sub(r'^\s*\w\s*', '', text)

    text = re.sub(r'\s+', ' ', text).strip()
   
    text = text.replace("b'", "").replace("'", "")
   
    text = text.lower()
    return text


## Now let's work on removing stopwords
Remove the stopwords.

In [88]:
# Your code

nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
 
    filtered_text = ' '.join(word for word in text.split() if word not in stop_words)
    return filtered_text



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [89]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [90]:
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [91]:
# Your code
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):

    words = text.split()
  
    lemmatized_text = ' '.join(lemmatizer.lemmatize(word) for word in words)
    return lemmatized_text


In [92]:

X_train_final_lemmatized = X_train_final_no_stopwords.apply(lemmatize_text)
X_test_final_lemmatized = X_test_final_no_stopwords.apply(lemmatize_text)
print(X_train_final_lemmatized.head())

29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I able reach oscar am. We supposed send pdb 11...
695    ; Huma Abedin B6I'm checking Pat 50k work Jack...
557                     I announced Monday - can't today
836    BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...
Name: text, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [93]:
# Your code
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Sample data
data = pd.DataFrame({
    'text': ["This is a sample sentence.", "Another example sentence for testing."],
    'label': [1, 0]
})

# Splitting the data into training and test sets
X = data['text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer on the training data and transform both training and test data
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Convert the result to a DataFrame for easier visualization
X_train_bow_df = pd.DataFrame(X_train_bow.toarray(), columns=vectorizer.get_feature_names_out())
X_test_bow_df = pd.DataFrame(X_test_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Display the Bag of Words representation
print("Training Data BoW:")
print(X_train_bow_df.head())

print("\nTest Data BoW:")
print(X_test_bow_df.head())


Training Data BoW:
   is  sample  sentence  this
0   1       1         1     1

Test Data BoW:
   is  sample  sentence  this
0   0       0         1     0


## Extra features

In [94]:

data_train = pd.DataFrame({
    'preprocessed_text': ["This is a test with euro and dollar.", "Another example with free money and transfer."]
})

data_val = pd.DataFrame({
    'preprocessed_text': ["Bank account and transaction.", "Cheap deals and win big!"]
})

In [95]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()


,preprocessed_text,money_mark,suspicious_words,text_len
0,This is a test with euro and dollar.,1,0,36
1,Another example with free money and transfer.,1,1,45


## How would work the Bag of Words with Count Vectorizer concept?

In [96]:
# Your code
data = pd.DataFrame({
    'text': ["This is a sample sentence.", "Another example sentence for testing.", "Free money in your bank account.", "Transfer funds to win big prizes!"]
})

# Splitting the data into training and test sets
X_train, X_test = train_test_split(data['text'], test_size=0.2, random_state=42)

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer on the training data and transform both training and test data
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Convert the result to a DataFrame for easier visualization
X_train_bow_df = pd.DataFrame(X_train_bow.toarray(), columns=vectorizer.get_feature_names_out())
X_test_bow_df = pd.DataFrame(X_test_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Display the Bag of Words representation
print("Training Data BoW:")
print(X_train_bow_df.head())

print("\nTest Data BoW:")
print(X_test_bow_df.head())


Training Data BoW:
   account  bank  big  free  funds  in  is  money  prizes  sample  sentence  \
0        0     0    1     0      1   0   0      0       1       0         0   
1        0     0    0     0      0   0   1      0       0       1         1   
2        1     1    0     1      0   1   0      1       0       0         0   

   this  to  transfer  win  your  
0     0   1         1    1     0  
1     1   0         0    0     0  
2     0   0         0    0     1  

Test Data BoW:
   account  bank  big  free  funds  in  is  money  prizes  sample  sentence  \
0        0     0    0     0      0   0   0      0       0       0         1   

   this  to  transfer  win  your  
0     0   0         0    0     0  


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [97]:
# Your code

data = pd.DataFrame({
    'text': [
        "This is a sample sentence.",
        "Another example sentence for testing.",
        "Free money in your bank account.",
        "Transfer funds to win big prizes!"
    ]
})

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the entire dataset and transform it into TF-IDF features
tfidf_matrix = vectorizer.fit_transform(data['text'])

# Print the shape of the vectorized dataset
print("Shape of the vectorized dataset:", tfidf_matrix.shape)


Shape of the vectorized dataset: (4, 20)


## And the Train a Classifier?

In [ ]:
# Your code

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, data['label'], test_size=0.2, random_state=42)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(report)


Accuracy: 0.0

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).